In [3]:
import json
import random
import pandas as pd

import mo_sql_parsing

import numpy as np
from tqdm import tqdm

ModuleNotFoundError: No module named 'mo_sql_parsing'

In [6]:
def is_small_select(query_info):
    if query_info.select['columns_num'] <= 2 and \
            query_info.select['*'] is False and \
            query_info.flags['groupby'] is False and \
            query_info.flags['having'] is False and \
            query_info.flags['joins'] is False and \
            query_info.flags['aggregates'] is False and \
            query_info.flags['distinct'] is False and \
            query_info.flags['orderby'] is False:
        return True
    return False


def is_large_select(query_info):
    if (query_info.select['columns_num'] > 2 or query_info.select['*'] is True) and\
            query_info.flags['groupby'] is False and \
            query_info.flags['having'] is False and \
            query_info.flags['joins'] is False and \
            query_info.flags['aggregates'] is False and \
            query_info.flags['distinct'] is False and \
            query_info.flags['orderby'] is False:
        return True
    return False


def is_aggregates(query_info):
    if query_info.flags['select'] is True and \
            query_info.flags['groupby'] is False and \
            query_info.flags['having'] is False and \
            query_info.flags['joins'] is False and \
            query_info.flags['aggregates'] is True and \
            query_info.flags['distinct'] is False and \
            'select' in query_info.aggregates.where:
        return True
    return False


def is_aggregates_and_group_by(query_info):
    if query_info.flags['select'] is True and \
            query_info.flags['groupby'] is True and \
            query_info.flags['joins'] is False and \
            query_info.flags['aggregates'] is True and \
            query_info.flags['distinct'] is False:
        return True
    return False


def is_join(query_info):
    if query_info.flags['select'] is True and \
            query_info.flags['groupby'] is False and \
            query_info.flags['having'] is False and \
            query_info.flags['joins'] is True and \
            query_info.flags['aggregates'] is False and \
            query_info.flags['distinct'] is False:
        return True
    return False


def is_join_and_aggregate(query_info):
    if query_info.flags['select'] is True and \
            query_info.flags['having'] is False and \
            query_info.flags['joins'] is True and \
            query_info.flags['aggregates'] is True and \
            query_info.flags['distinct'] is False and \
            'select' in query_info.aggregates.where:
        return True
    return False

In [3]:
def categorize_qr2t_benchmark(spider_datapoints):
    categories = [
        ["small_select", is_small_select],
        ["large_select", is_large_select],
        ["aggregate", is_aggregates],
        ["aggregate_group_by", is_aggregates_and_group_by],
        ["join", is_join],
        ["join_aggregate", is_join_and_aggregate]
    ]

    def find_category(query_info):
        for category, cond in categories:
            if cond(query_info):
                return category
        return None

    datapoints_with_category = []
    for datapoint in tqdm(spider_datapoints):
        try:
            cat = find_category(QueryInfo(datapoint['query']))
        except (ExtractException, ParseException):
            continue
        if cat is not None:
            datapoint['category'] = cat
            datapoints_with_category.append(datapoint)

    return datapoints_with_category

/home/mikexydas/PycharmProjects/Data2Text/notebooks
